In [1]:
import requests 
from bs4 import BeautifulSoup
import re

In [8]:
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import os

chromedriver = "/home/ubuntu/GCScrapper/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver


In [3]:
import json
coData = []
with open('SJ_salaries.json', 'r') as f:
    for line in f:
        q = json.dumps(line, encoding='utf8')
        jfile = json.loads(q)
        

In [4]:
x = json.loads(jfile[:161505])

In [5]:
#x = json.loads(jfile[:161505])
googleppl = x[x.keys()[0]].keys()

In [ ]:
len(googleppl)

In [6]:
# collect one linkedin url for each jorb
from pygoogle import pygoogle

In [9]:
import time
chrome_options = Options()
chrome_options.add_extension('/home/ubuntu/GCScrapper/hook.crx')

browser = webdriver.Chrome(chromedriver, chrome_options=chrome_options)

KeyboardInterrupt: 

In [ ]:
# for each linkedin profile
def dump_profiles(urls, pos, browser):
    profiles = []
    for url in urls:
        browser.get(url)
        time.sleep(1)
        try:
            ent = browser.find_element_by_tag_name("body")
            profile = ent.find_element_by_id("background").get_attribute('innerHTML')
            profiles.append(profile)
        except:
            continue

        #for each title, save company, title, and array of profiles to disk
    pdict = {'company': 'Google', 'position': pos, 'profiles': profiles}
    parse_li(pdict)

In [ ]:
pdict = {'company': 'Google', 'position': googleppl[0], 'profiles': profiles}
with open('SJ_profiles.json', 'a') as f:
    json.dump(pdict, f)

In [26]:
li_urls = []
for goog in googleppl[321:330]:
    search = "https://www.linkedin.com/title/ " + goog + " at Google"
    g = pygoogle(search)
    g.pages = 1
    li_url = g.get_urls()[0]
    time.sleep(5)

    r = requests.get(li_url)
    profiles = BeautifulSoup(r.content).findAll('li',{'class': 'item-container'})
    urls = []
    for p in profiles:
        href = p.find('a')['href']
        urls.append(href)
    li_urls.append({'position':goog, 'company':'Google', 'profiles': urls})
    # get these later
    #dump_profiles(urls, goog, browser)

pygoogle ERROR __search__| responseDetails : Suspected Terms of Service Abuse. Please see http://code.google.com/apis/errors
pygoogle ERROR __search__| responseDetails : Suspected Terms of Service Abuse. Please see http://code.google.com/apis/errors
pygoogle ERROR __search__| responseDetails : Suspected Terms of Service Abuse. Please see http://code.google.com/apis/errors
pygoogle ERROR __search__| responseDetails : Suspected Terms of Service Abuse. Please see http://code.google.com/apis/errors
pygoogle ERROR __search__| responseDetails : Suspected Terms of Service Abuse. Please see http://code.google.com/apis/errors
pygoogle ERROR __search__| responseDetails : Suspected Terms of Service Abuse. Please see http://code.google.com/apis/errors
pygoogle ERROR __search__| responseDetails : Suspected Terms of Service Abuse. Please see http://code.google.com/apis/errors
pygoogle ERROR __search__| responseDetails : Suspected Terms of Service Abuse. Please see http://code.google.com/apis/errors


ERROR:pygoogle:responseDetails : Suspected Terms of Service Abuse. Please see http://code.google.com/apis/errors


pygoogle INFO get_urls| No results returned
pygoogle INFO get_urls| No results returned
pygoogle INFO get_urls| No results returned
pygoogle INFO get_urls| No results returned
pygoogle INFO get_urls| No results returned
pygoogle INFO get_urls| No results returned
pygoogle INFO get_urls| No results returned
pygoogle INFO get_urls| No results returned
pygoogle INFO get_urls| No results returned
pygoogle INFO get_urls| No results returned
pygoogle INFO get_urls| No results returned
pygoogle INFO get_urls| No results returned
pygoogle INFO get_urls| No results returned
pygoogle INFO get_urls| No results returned
pygoogle INFO get_urls| No results returned
pygoogle INFO get_urls| No results returned
pygoogle INFO get_urls| No results returned
pygoogle INFO get_urls| No results returned
pygoogle INFO get_urls| No results returned
pygoogle INFO get_urls| No results returned
pygoogle INFO get_urls| No results returned
pygoogle INFO get_urls| No results returned
pygoogle INFO get_urls| No resul

INFO:pygoogle:No results returned


IndexError: list index out of range

In [ ]:
googleppl.index(goog)

In [23]:
li_urls

[{'company': 'Google', 'position': u'Software Developer', 'profiles': []},
 {'company': 'Google',
  'position': u'Merchandising Manager - Hourly',
  'profiles': []},
 {'company': 'Google', 'position': u'Senior', 'profiles': []},
 {'company': 'Google',
  'position': u'Vice President Marketing and Business Development',
  'profiles': []},
 {'company': 'Google',
  'position': u'Online Sales and Operations Specialist',
  'profiles': []},
 {'company': 'Google', 'position': u'Python Developer', 'profiles': []},
 {'company': 'Google',
  'position': u'Senior Account Planner Intern',
  'profiles': []},
 {'company': 'Google',
  'position': u'Software Quality Assurance Engineering Intern - Monthly',
  'profiles': []},
 {'company': 'Google',
  'position': u'Senior Technical Writer - Hourly Contractor',
  'profiles': []}]

In [ ]:
#parse linkedin json so it's not so onerous
# one document per person

def parse_li(x):
    boxes =['background-summary-container','background-experience-container',
        'background-languages-container', 'background-skills-container',
       'background-education-container','background-honors-container']
    cleanprofiles = []
    for p in x['profiles']:
        content = ""
        for b in boxes:
            try:
                divs = BeautifulSoup(p).find('div', {'id': b})
                for script in divs(["script", "style"]):
                    script.extract()    # rip it out   
                for d in divs.findAll('div'):
                    content += " ".join(d.strings) + " "
            except:
                continue
        cleanprofiles.append(content)
    x['profiles']=cleanprofiles
    x['position'].replace("/","-")
    with open('data/'+x['position']+'.json','w') as f:
        json.dump(x, f)